# Script para a inserção das entidades PEPs no banco


In [2]:
import pandas as pd
import os, sys, django, neomodel, dotenv, wikipedia
from datetime import datetime
import locale
from bs4 import BeautifulSoup
# from wiki import Wiki
# from database_manager import DatabaseManager
locale.setlocale(locale.LC_TIME, "pt_BR.UTF-8")

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(BASE_DIR)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "tcc.settings")
django.setup()
dotenv.load_dotenv()

from myapi.models import Pessoa, Organizacao

In [3]:
wikipedia.set_lang("pt")
pagina_wiki = wikipedia.page(title='jair bolsonaro')

In [4]:
soup = BeautifulSoup(pagina_wiki.html(), 'html.parser')
infobox = soup.find('table', {'class': 'infobox'})
infobox

<table class="infobox vcard" style="width:22em;"><tbody><tr><th class="infobox-above infobox-above" colspan="2" style="font-size:100%;"><span><div class="fn" style="font-size:125%;">Jair Bolsonaro</div></span></th></tr><tr><td class="infobox-image" colspan="2"><span typeof="mw:File"><a class="mw-file-description" href="/wiki/Ficheiro:Jair_Bolsonaro_2019_Portrait_(3x4_cropped_center).jpg"><img class="mw-file-element" data-file-height="2158" data-file-width="1666" decoding="async" height="337" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/37/Jair_Bolsonaro_2019_Portrait_%283x4_cropped_center%29.jpg/330px-Jair_Bolsonaro_2019_Portrait_%283x4_cropped_center%29.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/37/Jair_Bolsonaro_2019_Portrait_%283x4_cropped_center%29.jpg/500px-Jair_Bolsonaro_2019_Portrait_%283x4_cropped_center%29.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/3/37/Jair_Bolsonaro_2019_Portrait_%283x4_cropped_center%29.jpg/960px-Jair_Bolsonaro_2019